In [1]:
import json

with open('credentials.json') as file:
    credentials = json.load(file)

In [2]:
import requests
from datetime import datetime, timedelta

def get_sessions():
    '''
    Take a learning map and returns the associated sessions in a json object.
    '''

    url = "https://events.rainfocus.com/api/search"

    payload =   {'search.sessiontype': 'Technical_seminar',
                'type': 'session',
                'catalogDisplay': 'list',
                'browserTimezone': 'Europe%2FParis'}

    headers = {
    'authority': 'events.rainfocus.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,fr-FR;q=0.8,fr;q=0.7',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'origin': 'https://www.ciscolive.com',
    'referer': 'https://www.ciscolive.com/',
    'rfapiprofileid': credentials['rfapiprofileid'],
    'rfauthtoken': credentials['rfauthtoken'],
    'rfwidgetid': credentials['rfwidgetid'],
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response.json()

class Session():

    def __init__(self, session_json):
        self.type = session_json['type']
        try:
            self.start = datetime.strptime(session_json['times'][0]['utcStartTime'], '%Y/%m/%d %H:%M:%S') + timedelta(hours=1)
            self.end = datetime.strptime(session_json['times'][0]['utcEndTime'], '%Y/%m/%d %H:%M:%S') + timedelta(hours=1)
        except KeyError:
            self.start = 'Null'
            self.end = 'Null'
        self.name = session_json['title']
        self.id = session_json['code']
        match int(self.id.split('-')[1][0]):
            case 1: self.level = 'Introductory'
            case 2: self.level = 'Intermediate'
            case 3: self.level = 'Advanced'
            case 4: self.level = 'General'
        self.capacity = int(session_json['times'][0]['capacity'])
        self.seats_remaining = int(session_json['times'][0]['seatsRemaining'])
        self.waitlist_remaining = int(session_json['times'][0]['waitlistRemaining'])
        
    def __str__(self):
        return self.name

In [3]:
sessions_json = get_sessions()

In [4]:
sessions = []

for session in sessions_json['sectionList'][0]['items']:
    sessions.append(Session(session))

In [5]:
for session in sessions:
    print(session.id, 'capacity =', session.capacity, 'empty seats =', session.seats_remaining)

TECSPG-3204 capacity = 69 empty seats = 14
TECTRS-3477 capacity = 152 empty seats = 81
TECARC-2407 capacity = 162 empty seats = 122
TECDCN-2495 capacity = 205 empty seats = 170
TECOPS-2004 capacity = 205 empty seats = 114
TECARC-2446 capacity = 205 empty seats = 126
TECENS-2618 capacity = 57 empty seats = 7
TECCRT-3005 capacity = 38 empty seats = 0
TECCRT-3000 capacity = 60 empty seats = 10
TECSPG-2014 capacity = 66 empty seats = 38
TECCLD-1000 capacity = 24 empty seats = 15
TECSEC-3780 capacity = 162 empty seats = 117
TECSEC-2780 capacity = 30 empty seats = 16
TECCLD-2307 capacity = 30 empty seats = 10
TECCOL-2190 capacity = 69 empty seats = 15
TECENT-3377 capacity = 176 empty seats = 9
TECEWN-2004 capacity = 30 empty seats = 0
TECENS-2001 capacity = 44 empty seats = 2
TECIOT-2584 capacity = 176 empty seats = 131
TECGRN-2760 capacity = 30 empty seats = 21
TECCRT-3778 capacity = 57 empty seats = 13
TECOPS-2005 capacity = 30 empty seats = 2
TECCLD-2394 capacity = 24 empty seats = 12
TEC

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(columns=['Session Name', 'Total Capacity', 'Empty Seats', '% Available'])

for session in sessions:
    to_append = {
        'Session Name': session.id + ' - ' + session.name,
        'Total Capacity': session.capacity,
        'Empty Seats': session.seats_remaining, 
        '% Available': int((session.seats_remaining / session.capacity) * 100)
    }
    df = df.append(to_append, ignore_index=True)

/var/folders/mg/zpzk185145107_n259csjmd40000gn/T/ipykernel_64056/3793009335.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(to_append, ignore_index=True)
/var/folders/mg/zpzk185145107_n259csjmd40000gn/T/ipykernel_64056/3793009335.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(to_append, ignore_index=True)
/var/folders/mg/zpzk185145107_n259csjmd40000gn/T/ipykernel_64056/3793009335.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(to_append, ignore_index=True)
/var/folders/mg/zpzk185145107_n259csjmd40000gn/T/ipykernel_64056/3793009335.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

In [8]:
df

,Session Name,Total Capacity,Empty Seats,% Available
0,TECSPG-3204 - Advanced ASR 9000 Operation and ...,69,14,20
1,TECTRS-3477 - Advanced Troubleshooting SD-WAN,152,81,53
2,"TECARC-2407 - Architecture, Deployments, and T...",162,122,75
3,TECDCN-2495 - Building Clouds for Enterprise A...,205,170,82
4,TECOPS-2004 - Campus Automation: How to Design...,205,114,55
5,TECARC-2446 - Catalyst 9000 Switching Family A...,205,126,61
6,TECENS-2618 - Catalyst 9000 Switching Software...,57,7,12
7,TECCRT-3005 - CCDE Techtorial,38,0,0
8,TECCRT-3000 - CCIE Enterprise Infrastructure T...,60,10,16
9,TECSPG-2014 - Cisco Converged SDN Transport,66,38,57


In [9]:
df.to_excel('open_tech_seminar.xlsx', index=False)